In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG19, ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_dir = '/content/drive/My Drive/cassava_leaf_disease/train'
test_dir = '/content/drive/My Drive/cassava_leaf_disease/test'


In [ ]:
image_size = (224, 224)
batch_size = 32

In [ ]:
# Data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,             # Rescale pixel values to [0, 1]
    shear_range=0.2,            # Shear transformations
    zoom_range=0.2,             # Zoom transformations
    horizontal_flip=True       # Horizontal flips
)

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:

# Load the training and test datasets
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

In [ ]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)


In [ ]:
# VGG19 Model
def build_vgg19():
    # Load pre-trained VGG19 model with ImageNet weights
    base_model = VGG19(input_shape=(image_size[0], image_size[1], 3), weights='imagenet', include_top=False)

    # Freeze base model layers to prevent training
    for layer in base_model.layers:
        layer.trainable = False

    # Flatten the output of the base model
    x = Flatten()(base_model.output)

    # Add a dense layer with ReLU activation
    x = Dense(512, activation='relu')(x)

    # Output layer with softmax activation for classification
    outputs = Dense(len(train_generator.class_indices), activation='softmax')(x)

    # Construct the model
    model = Model(inputs=base_model.input, outputs=outputs)
    return model


In [ ]:
# Compile and train VGG19
vgg19_model = build_vgg19()
vgg19_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
vgg19_history = vgg19_model.fit(train_generator, validation_data=test_generator, epochs=5)

RestNet50

In [ ]:
# ResNet50 Model
def build_resnet50():
    # Load pre-trained ResNet50 model with ImageNet weights
    base_model = ResNet50(input_shape=(image_size[0], image_size[1], 3), weights='imagenet', include_top=False)

    # Freeze base model layers to prevent training
    for layer in base_model.layers:
        layer.trainable = False

    # Flatten the output of the base model
    x = Flatten()(base_model.output)

    # Add a dense layer with ReLU activation
    x = Dense(512, activation='relu')(x)

    # Output layer with softmax activation for classification
    outputs = Dense(len(train_generator.class_indices), activation='softmax')(x)

    # Construct the model
    model = Model(inputs=base_model.input, outputs=outputs)
    return model


In [ ]:
# Compile and train ResNet50
resnet50_model = build_resnet50()
resnet50_model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
resnet50_history = resnet50_model.fit(train_generator, validation_data=test_generator, epochs=5)


In [ ]:
from tensorflow.keras.applications import InceptionV3

# InceptionV3 Model
def build_inceptionv3():
    # Load pre-trained InceptionV3 model with ImageNet weights
    base_model = InceptionV3(input_shape=(image_size[0], image_size[1], 3), weights='imagenet', include_top=False)

    # Freeze base model layers to prevent training
    for layer in base_model.layers:
        layer.trainable = False

    # Flatten the output of the base model
    x = Flatten()(base_model.output)

    # Add a dense layer with ReLU activation
    x = Dense(512, activation='relu')(x)

    # Output layer with softmax activation for classification
    outputs = Dense(len(train_generator.class_indices), activation='softmax')(x)

    # Construct the model
    model = Model(inputs=base_model.input, outputs=outputs)
    return model

In [ ]:
# Compile and train InceptionV3
inceptionv3_model = build_inceptionv3()
inceptionv3_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
inceptionv3_history = inceptionv3_model.fit(train_generator, validation_data=test_generator, epochs=5)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

# Function to plot training history
def plot_training_history(history, metric='accuracy'):
    plt.plot(history.history[metric])
    plt.plot(history.history['val_' + metric])
    plt.title('Model ' + metric)
    plt.ylabel(metric)
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()


In [ ]:
# Plot training history for VGG19
plot_training_history(vgg19_history, metric='accuracy')
plot_training_history(vgg19_history, metric='loss')

In [ ]:
# Plot training history for ResNet50
plot_training_history(resnet50_history, metric='accuracy')
plot_training_history(resnet50_history, metric='loss')

In [ ]:
# Plot training history for InceptionV3
plot_training_history(inceptionv3_history, metric='accuracy')
plot_training_history(inceptionv3_history, metric='loss')

In [ ]:
# Function to evaluate model
def evaluate_model(model, test_generator):
    # Predict labels
    y_pred = model.predict(test_generator)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true = test_generator.classes

    # Accuracy
    accuracy = accuracy_score(y_true, y_pred_classes)
    print('Accuracy:', accuracy)

    # Precision
    precision = precision_score(y_true, y_pred_classes, average='weighted')
    print('Precision:', precision)

    # Recall
    recall = recall_score(y_true, y_pred_classes, average='weighted')
    print('Recall:', recall)

    # F1 score
    f1 = f1_score(y_true, y_pred_classes, average='weighted')
    print('F1 Score:', f1)

    # Classification report
    print('Classification Report:')
    print(classification_report(y_true, y_pred_classes))

    # Confusion matrix
    print('Confusion Matrix:')
    print(confusion_matrix(y_true, y_pred_classes))

In [ ]:
# Evaluate VGG19 model
print('Evaluation for VGG19 model:')
evaluate_model(vgg19_model, test_generator)

In [ ]:
# Evaluate ResNet50 model
print('Evaluation for ResNet50 model:')
evaluate_model(resnet50_model, test_generator)

In [ ]:
# Evaluate InceptionV3 model
print('Evaluation for InceptionV3 model:')
evaluate_model(inceptionv3_model, test_generator)